<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/multiverse_topology_prediction_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
multiverse_topology_prediction.py

Train a neural network to predict multiverse topology classes
from string-theory compactification parameters.
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# 1. Model Definition
class MultiverseAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dim=32, output_dim=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        logits = self.net(x)
        return F.softmax(logits, dim=-1)

# 2. Synthetic Dataset Generation
def sample_compactifications(n_samples=10000, input_dim=5):
    """
    Generate synthetic compactification parameters and discrete topology labels.
    """
    params = np.random.uniform(0, 2*np.pi, size=(n_samples, input_dim)).astype(np.float32)
    scores = np.sum(np.sin(params), axis=1)
    # Use np.ptp() instead of ndarray.ptp(), added 1e-6 to avoid division by zero
    labels = np.floor((scores - scores.min()) / (np.ptp(scores) + 1e-6) * 10).astype(int)
    labels = np.clip(labels, 0, 9)
    return params, labels

# 3. Training and Validation Function
def train_and_validate(
    model, train_loader, val_loader, epochs=50, lr=1e-3, device=None
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        # Training
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * xb.size(0)
            correct_train += (preds.argmax(1) == yb).sum().item()
            total_train += xb.size(0)

        train_loss = running_loss / total_train
        train_acc  = correct_train / total_train

        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                correct_val += (preds.argmax(1) == yb).sum().item()
                total_val += xb.size(0)
        val_acc = correct_val / total_val

        if epoch == 1 or epoch % 10 == 0:
            print(
                f"Epoch {epoch:02d} ─ "
                f"Train Loss: {train_loss:.4f} ─ "
                f"Train Acc: {train_acc:.3f} ─ "
                f"Val Acc: {val_acc:.3f}"
            )

# 4. Main Execution
if __name__ == "__main__":
    # Hyperparameters
    INPUT_DIM   = 5
    HIDDEN_DIM  = 32
    OUTPUT_DIM  = 10
    N_SAMPLES   = 10000
    TEST_SIZE   = 0.2
    BATCH_SIZE  = 128
    EPOCHS      = 50
    LEARNING_RT = 1e-3

    # Generate data
    X, y = sample_compactifications(n_samples=N_SAMPLES, input_dim=INPUT_DIM)

    # Split into training and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=42
    )

    # Create DataLoaders
    train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    val_ds   = TensorDataset(torch.from_numpy(X_val),   torch.from_numpy(y_val))
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

    # Initialize and train model
    model = MultiverseAI(input_dim=INPUT_DIM,
                         hidden_dim=HIDDEN_DIM,
                         output_dim=OUTPUT_DIM)
    train_and_validate(
        model, train_loader, val_loader,
        epochs=EPOCHS, lr=LEARNING_RT
    )

    # Sample evaluation
    sample_params = np.random.uniform(0, 2*np.pi, size=(3, INPUT_DIM)).astype(np.float32)
    with torch.no_grad():
        probs = model(torch.from_numpy(sample_params))
    print("\nSample Predictions:")
    for i, p in enumerate(probs):
        print(f"  Params {i+1}: {p.numpy()}")